In [ ]:
import pickle
import numpy as np
import sympy
import pandas as pd
import sys
import time
import cartpole_error_monitoring

def load_from_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
        F = data['F']
        lmd_time_series = data['lmd_time_series']
        state_time_series = data['state_time_series']
    return F, lmd_time_series, state_time_series


F, lmd_time_series, state_time_series= load_from_file('./log/symbolic_F.pkl')


In [ ]:
for i in range(F.shape[1]):
    variables = set()
    matrix = F[i,0]
    variables.update(matrix.free_symbols)
    # print(list(variables))

for i in range(state_time_series.shape[0]):
    for j in range(state_time_series.shape[1]):
        variables = set()
        matrix = state_time_series[i,j]
        variables.update(matrix.free_symbols)
        # print(list(variables))

for i in range(lmd_time_series.shape[0]):
    for j in range(lmd_time_series.shape[1]):
        variables = set()
        matrix = lmd_time_series[i,j]
        variables.update(matrix.free_symbols)
        # print(list(variables))


In [ ]:
project_name = "cartpole_error_monitoring"
# 読み込みたいファイル名のリスト
filenames = ['./log/'+project_name+'_uopt.log', './log/'+project_name+'_x.log', './log/'+project_name+'_t.log']

# 各ファイルを読み込み、その結果をリストに保存します
df_list = [pd.read_csv(filename, delim_whitespace=True, header=None) for filename in filenames]

# 各データフレームをnumpyの行列に変換し、それをリストに保存します
u_matrix, x_matrix, t = [df.values for df in df_list]


In [ ]:
import os, json
from math import sqrt

from matplotlib import pyplot as plt

config_name = 'config_20230722.json'

with open('./config/'+config_name, 'r') as f:
    config = json.load(f)

# Create and configure the AutoGenU object
auto_gen_u = cartpole_error_monitoring.create_autogenu_object(config)
cartpole_error_monitoring.set_parameters(auto_gen_u, config)


x = sympy.Matrix(sympy.symbols(f'x0:{config["nx"]}'))  # Initial state as variables
u = sympy.Matrix([sympy.symbols(f'u{i}_{j}') for i in range(config["nu"]) for j in range(config["N"])]).reshape(config["nu"], config["N"])
r = sympy.symbols('r[0:%d]' %(config["nu"]))
q_terminal = sympy.symbols('q_terminal[0:%d]' %(config["nx"]))
q = sympy.symbols('q[0:%d]' %(config["nx"]))
x_ref = sympy.symbols('x_ref[0:%d]' %(config["nx"]))


sub_param = auto_gen_u.make_subs_dicts()

# Create a list to store the L2 norms and F values
l2_norms = []
F_param_substituted = []

simulation_steps = int(config["simulation_length"]/config["sampling_time"])
print("simulation step:", simulation_steps)
step_num = 3
start_time = time.time()
# for k in range(int(config["simulation_length"]/config["sampling_time"])):
for step in range(step_num):
    print(step)
    # Create the substitutions for the state and input at the current time step
    subs_x = [(x[l], x_matrix[step,l]) for l in range(config["nx"])]
    subs_u = [(u[l], u_matrix[step,l]) for l in range(config["N"])]
    print(f"x[0] = {x_matrix[step,:]}")
    print(f"U = {subs_u}")

    # Create a list to store the F values for the current time step
    F_param_substituted_step = []

    # Loop over the control inputs
    norm = 0
    for N in range(config["N"]):
        for j in range(config["nu"]):
            # Calculate the value of F
            F_val = F[N,j].subs(sub_param).subs(subs_x).subs(subs_u)
            F_param = F[N,j].subs(sub_param)
            F_param_substituted_step.append(F_param)
            
            # Add to the squared L2 norm
            norm += F_val**2
            print(f"F_val = {F_val}")
    
    # Append the square root of the norm (i.e., the L2 norm) to the list
    l2_norms.append(sqrt(norm))
    print(sqrt(norm))
    end_time = time.time()
    print("computation time in %d step: %f" % (k, end_time - start_time))


    # Append the F values for the current step to the list
    # F_param_substituted.append(F_param_substituted_step)

simulation_steps = int(config["simulation_length"]/config["sampling_time"])
t_series = [item for sublist in t[0:step_num] for item in sublist]
result = np.log10(l2_norms)
plt.plot(t_series,result)
plt.ylabel(r'$\log_{10}\Vert F \Vert_2$')
plt.xlabel('Time (s)')